<a href="https://colab.research.google.com/github/seohyun126/LG-Aimers-6th/blob/main/Aimers_%EB%AA%A8%EB%8D%B8%EB%A7%81_0%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **✅ Random Forest + HyperOpt + Stratified K-Fold CV + ROC-AUC 평가**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
train=pd.read_csv('/content/drive/MyDrive/Aimers_data/train_1.csv',index_col=False)
test=pd.read_csv('/content/drive/MyDrive/Aimers_data/test_1.csv',index_col=False)
sub=pd.read_csv("/content/drive/MyDrive/Aimers_data/sample_submission.csv",index_col=False)

In [ ]:
train.drop('Unnamed: 0',axis=1,inplace=True)
test.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
train.head()

,시술 유형,배란 자극 여부,단일 배아 이식 여부,착상 전 유전 진단 사용 여부,남성 주 불임 원인,남성 부 불임 원인,여성 주 불임 원인,여성 부 불임 원인,부부 주 불임 원인,부부 부 불임 원인,...,현재 시술용 여부,난자 기증 여부,정자 기증 여부,특정 시술 유형_GIFT,특정 시술 유형_ICI,특정 시술 유형_ICSI,특정 시술 유형_IUI,특정 시술 유형_IVF,특정 시술 유형_Unknown,시술 나이
0,1,1.0,0.0,0.0,0,0,0,0,0,0,...,1.0,0.5,1.0,0,0,1,0,0,0.0,1.0
1,1,0.0,0.0,0.0,0,0,0,0,0,0,...,1.0,0.5,1.0,0,0,1,0,0,0.0,6.0
2,1,1.0,0.0,0.0,0,0,0,0,0,0,...,1.0,0.5,1.0,0,0,0,0,1,0.0,1.0
3,1,1.0,0.0,0.0,0,0,0,0,0,0,...,1.0,0.5,1.0,0,0,1,0,0,0.0,2.0
4,1,1.0,0.0,0.0,0,0,0,0,0,0,...,1.0,0.5,1.0,0,0,1,0,0,0.0,1.0


In [ ]:
train.shape

(256201, 63)

In [ ]:
test.shape

(90067, 61)

In [ ]:
train.drop('특정 시술 유형_GIFT',axis=1,inplace=True)

In [ ]:
!pip install hyperopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 12.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from hyperopt import hp, fmin, tpe, Trials
from sklearn.metrics import roc_auc_score, make_scorer

#  데이터 준비
X = train.drop('임신 성공 여부',axis=1,inplace=False)
y = train['임신 성공 여부']  # 타겟 변수

In [ ]:
# Stratified K-Fold 설정 -> 불균형 데이터 고려한 것!
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# HyperOpt 하이퍼파라미터 검색 공간 정의
space = {
    'n_estimators': hp.choice('n_estimators', range(50, 500, 50)),  # 50~450
    'max_depth': hp.choice('max_depth', range(3, 20)),  # 3~19
    'min_samples_split': hp.uniform('min_samples_split', 0.01, 0.2),  # 1%~20%
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0.01, 0.2),  # 1%~20%
    'class_weight': hp.choice('class_weight', [None, 'balanced', 'balanced_subsample'])  # 불균형 데이터 고려
}

# 목적 함수 정의 (ROC-AUC 평가)
def objective(params):
    model = RandomForestClassifier(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        min_samples_split=params['min_samples_split'],
        min_samples_leaf=params['min_samples_leaf'],
        class_weight=params['class_weight'],
        random_state=42,
        n_jobs=-1
    )

    # Cross-Validation 평가 (Stratified K-Fold 사용)
    auc_scores = cross_val_score(model, X, y, cv=cv, scoring=make_scorer(roc_auc_score))

    # 평균 ROC-AUC 반환 (HyperOpt는 최소화 문제이므로 -1 곱함)
    return -np.mean(auc_scores)

#  HyperOpt 실행
trials = Trials()
best_params = fmin(
    fn=objective,  # 목적 함수
    space=space,   # 하이퍼파라미터 검색 공간
    algo=tpe.suggest,  # Tree-structured Parzen Estimator
    max_evals=30,  # 30번 시도
    trials=trials,
    verbose=True
)

  3%|▎         | 1/30 [00:10<05:05, 10.53s/trial, best loss: -0.5]


KeyboardInterrupt: 

In [ ]:
# 최적 하이퍼파라미터 출력
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'class_weight': 1, 'max_depth': 16, 'min_samples_leaf': 0.03213487829338136, 'min_samples_split': 0.01696029668885038, 'n_estimators': 8}


In [ ]:
best_params['class_weight']

1

In [ ]:
# 최적 파라미터로 최종 모델 학습
best_model = RandomForestClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

In [ ]:
best_params['min_samples_leaf']

0.028889211274696

In [ ]:
best_model.fit(X, y)

RandomForestClassifier(class_weight='balanced', max_depth=14,
                       min_samples_leaf=0.028889211274696,
                       min_samples_split=0.025966801478973345, n_estimators=6,
                       n_jobs=-1, random_state=42)

In [ ]:
# 최적 모델로 확률 예측 (ROC-AUC 기준이므로 확률 값 예측)
test_preds = best_model.predict_proba(test)[:, 1]  # 임신 성공 확률 (1일 확률)

In [ ]:
test['임신 성공 여부'] = test_preds  # 예측된 확률값 추가

In [ ]:
test['임신 성공 여부']

,임신 성공 여부
0,0.260899
1,0.262516
2,0.449918
3,0.391141
4,0.657737
...,...
90062,0.261542
90063,0.414290
90064,0.572893
90065,0.509719


In [ ]:
sub=pd.read_csv("/content/drive/MyDrive/Aimers_data/sample_submission.csv")

In [ ]:
sub

,ID,probability
0,TEST_00000,0.0
1,TEST_00001,0.0
2,TEST_00002,0.0
3,TEST_00003,0.0
4,TEST_00004,0.0
...,...,...
90062,TEST_90062,0.0
90063,TEST_90063,0.0
90064,TEST_90064,0.0
90065,TEST_90065,0.0


In [ ]:
sub['probability']=test['임신 성공 여부']

In [ ]:
sub

,ID,probability
0,TEST_00000,0.260899
1,TEST_00001,0.262516
2,TEST_00002,0.449918
3,TEST_00003,0.391141
4,TEST_00004,0.657737
...,...,...
90062,TEST_90062,0.261542
90063,TEST_90063,0.414290
90064,TEST_90064,0.572893
90065,TEST_90065,0.509719


In [ ]:
sub.to_csv("/content/drive/MyDrive/Aimers_data/sample_submission1.csv",index=False)

0.71349

In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 23.0 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
model = CatBoostClassifier(random_state=42, silent=True)
# GridSearchCV에서 사용할 파라미터 설정
param_grid = {
    'iterations': [100, 200, 300],  # 트리의 개수
    'depth': [4, 6, 8, 10],  # 트리의 깊이
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # 학습률
    'l2_leaf_reg': [1, 3, 5, 10],  # L2 규제 파라미터
    'border_count': [32, 64],  # 이진화 과정에서 사용할 경계의 수
}


# 3. StratifiedKFold로 불균형 데이터셋을 고려한 K-Fold 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 4. ROC-AUC를 평가 지표로 설정
roc_auc_scorer = make_scorer(roc_auc_score)

# 5. GridSearchCV 실행
grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                           scoring=roc_auc_scorer, cv=cv, n_jobs=-1, verbose=2)

In [ ]:
train.columns

Index(['시술 유형', '배란 자극 여부', '단일 배아 이식 여부', '착상 전 유전 진단 사용 여부', '남성 주 불임 원인',
       '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인',
       '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애',
       '불임 원인 - 여성 요인', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도',
       '불임 원인 - 정자 면역학적 요인', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '총 시술 횟수',
       '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수',
       'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수',
       '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수',
       '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수',
       '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '동결 배아 사용 여부',
       '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부',
       '배아 이식 경과일', '임신 성공 여부', '현재 시술용 여부', '난자 기증 여부', '정자 기증 여부',
       '특정 시술 유형_ICI', '특정 시술 유형_ICSI', '특정 시술 유형_IUI', '특정 시술 유형_IVF',
       '특정 시술 유형_Unknown', '시술 나이']

In [ ]:
test.shape

(90067, 62)

In [ ]:
grid_search.best_params_

In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
roc_auc_score(y,)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostClassifier


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
model = CatBoostClassifier(random_state=42, silent=True, task_type='CPU')

In [ ]:
param_dist = {
    'iterations': [100, 200],  # 트리의 개수 (간단한 범위로 축소)
    'depth': [2, 4, 6, 8, 10],  # 트리의 깊이
    'learning_rate': [0.01, 0.05, 0.005],  # 학습률
    'l2_leaf_reg': [1, 3],  # L2 규제 파라미터
}

# 3. StratifiedKFold로 불균형 데이터셋을 고려한 K-Fold 설정
cv = 3  # 교차 검증 횟수 줄이기

# 4. ROC-AUC를 평가 지표로 설정
roc_auc_scorer = make_scorer(roc_auc_score)

# 5. RandomizedSearchCV 실행
random_search = RandomizedSearchCV(
    estimator=model, param_distributions=param_dist,
    scoring=roc_auc_scorer, cv=cv, n_jobs=-1, verbose=2, n_iter=30  # 30개의 랜덤 조합 시도
)

# 모델 학습
random_search.fit(X, y)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


RandomizedSearchCV(cv=3,
                   estimator=<catboost.core.CatBoostClassifier object at 0x799a5077f390>,
                   n_iter=30, n_jobs=-1,
                   param_distributions={'depth': [2, 4, 6, 8, 10],
                                        'iterations': [100, 200],
                                        'l2_leaf_reg': [1, 3],
                                        'learning_rate': [0.01, 0.05, 0.005]},
                   scoring=make_scorer(roc_auc_score, response_method='predict'),
                   verbose=2)

In [ ]:
# 최적 파라미터 출력
print("Best parameters found: ", random_search.best_params_)
# 최적 모델
best_model = random_search.best_estimator_

Best parameters found:  {'learning_rate': 0.05, 'l2_leaf_reg': 1, 'iterations': 200, 'depth': 10}


In [ ]:
cat_test_preds = best_model.predict_proba(test)[:, 1]
test['임신 성공 여부'] = cat_test_preds

In [ ]:
test[test['임신 성공 여부']>0.5].shape[0]

4374

In [ ]:
test['시술 나이'].value_counts()

,count
시술 나이,
1.000,36178
2.000,20211
3.000,13611
4.000,13266
5.000,4272
6.000,2413
2.294,116


In [ ]:
test.loc[test['시술 나이']==2.294,'임신 성공 여부']=0.0001

In [ ]:
test[test['임신 성공 여부']>0.5].shape[0]

4374

In [ ]:
sub['probability']=test['임신 성공 여부']

In [ ]:
sub['probability']=sub['probability'].round(3)

In [ ]:
sub.head()

,ID,probability
0,TEST_00000,0.003
1,TEST_00001,0.017
2,TEST_00002,0.171
3,TEST_00003,0.115
4,TEST_00004,0.501


In [ ]:
sub.to_csv("/content/drive/MyDrive/Aimers_data/sample_submission2_2.csv",index=False)

In [ ]:
print(test[test['임신 성공 여부2']>0.5].shape[0])
print(test[test['임신 성공 여부']>0.5].shape[0])

3581
42868


In [ ]:
sub['probability']=test['임신 성공 여부2']

In [ ]:
sub

,ID,probability
0,TEST_00000,0.004690
1,TEST_00001,0.018120
2,TEST_00002,0.160244
3,TEST_00003,0.126281
4,TEST_00004,0.490076
...,...,...
90062,TEST_90062,0.004651
90063,TEST_90063,0.186509
90064,TEST_90064,0.494624
90065,TEST_90065,0.241246


In [ ]:
test.shape[0]

90067

In [ ]:
train['임신 성공 여부'].value_counts()

,count
임신 성공 여부,
0,190013
1,66188


In [ ]:
test.head()

,Unnamed: 0,시술 유형,배란 자극 여부,단일 배아 이식 여부,착상 전 유전 진단 사용 여부,남성 주 불임 원인,남성 부 불임 원인,여성 주 불임 원인,여성 부 불임 원인,부부 주 불임 원인,...,난자 기증 여부,정자 기증 여부,특정 시술 유형_ICI,특정 시술 유형_ICSI,특정 시술 유형_IUI,특정 시술 유형_IVF,특정 시술 유형_Unknown,시술 나이,임신 성공 여부,임신 성공 여부2
0,0,1,1.0,0.0,0.0,0,0,0,0,0,...,0.5,1.0,0,0,0,1,0.0,2.0,0.260899,0.008113
1,1,1,1.0,0.0,0.0,0,0,0,0,0,...,0.5,1.0,0,0,0,1,0.0,1.0,0.262516,0.028953
2,2,1,1.0,0.0,0.0,0,0,0,0,0,...,0.5,1.0,0,1,0,0,0.0,4.0,0.449918,0.157696
3,3,1,1.0,0.0,0.0,0,0,0,0,0,...,0.5,1.0,0,1,0,0,0.0,4.0,0.391141,0.131762
4,4,1,1.0,0.0,0.0,0,0,0,0,0,...,0.5,1.0,0,1,0,0,0.0,2.0,0.657737,0.485478


In [ ]:
test['임신 성공 여부 3']=test['임신 성공 여부2']*0.75 + test['임신 성공 여부']*0.25

In [ ]:
print(test[test['임신 성공 여부 3']>0.5].shape[0])
print(test[test['임신 성공 여부2']>0.5].shape[0])
print(test[test['임신 성공 여부']>0.5].shape[0])

8961
2499
42868


In [ ]:
sub['probability']=test['임신 성공 여부 3']

In [ ]:
sub

,ID,probability
0,TEST_00000,0.071310
1,TEST_00001,0.087344
2,TEST_00002,0.230751
3,TEST_00003,0.196607
4,TEST_00004,0.528543
...,...,...
90062,TEST_90062,0.068464
90063,TEST_90063,0.241601
90064,TEST_90064,0.508578
90065,TEST_90065,0.292864


In [ ]:
sub.to_csv("/content/drive/MyDrive/Aimers_data/sample_submission1_4.csv",index=False)

In [ ]:
sub.to_csv("/content/drive/MyDrive/Aimers_data/sample_submission1_2.csv",index=False)

In [ ]:
sub['probability']=(train['임신 성공 여부']+train['임신 성공 여부2'])/2